In [1]:
import numpy as np
import pandas as pd

In [2]:
training_data = pd.read_csv("train.csv")

VISUALISING THE DATA

In [3]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
training_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


CHECKING WHERE THE DATA IS NOT GIVEN

In [5]:
training_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
testing_data = pd.read_csv("test.csv")

In [7]:
testing_data.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [8]:
test_id = testing_data["PassengerId"]

In [9]:
test_id

0       892
1       893
2       894
3       895
4       896
       ... 
413    1305
414    1306
415    1307
416    1308
417    1309
Name: PassengerId, Length: 418, dtype: int64

REMOVING TICKET AND CABIN COLUMNS AS THEY ARE NOT IMPACTFUL
AND THE DATA IS ALSO NOT FULLY AVAILABLE

In [10]:
training_data = training_data.drop(["PassengerId","Name","Ticket","Cabin"],axis=1)
testing_data = testing_data.drop(["PassengerId","Name","Ticket","Cabin"],axis=1)

CONVERTING ALL DATA INTO NUMERICAL FORM

In [11]:
dataset = [training_data,testing_data]

In [12]:
for row in dataset:
    row['Sex'] = row['Sex'].map( {'female': -1, 'male': 1} ).astype(int)

In [13]:
testing_data['Embarked'] = testing_data['Embarked'].map( {'S': -1, 'C': 0,'Q':1} )
training_data['Embarked'] = training_data['Embarked'].map( {'S': -1, 'C': 0,'Q':1} )

In [14]:
training_data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,-1.0
1,1,1,-1,38.0,1,0,71.2833,0.0
2,1,3,-1,26.0,0,0,7.9250,-1.0
3,1,1,-1,35.0,1,0,53.1000,-1.0
4,0,3,1,35.0,0,0,8.0500,-1.0
...,...,...,...,...,...,...,...,...
886,0,2,1,27.0,0,0,13.0000,-1.0
887,1,1,-1,19.0,0,0,30.0000,-1.0
888,0,3,-1,NaN,1,2,23.4500,-1.0
889,1,1,1,26.0,0,0,30.0000,0.0


In [15]:
testing_data

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,34.5,0,0,7.8292,1
1,3,-1,47.0,1,0,7.0000,-1
2,2,1,62.0,0,0,9.6875,1
3,3,1,27.0,0,0,8.6625,-1
4,3,-1,22.0,1,1,12.2875,-1
...,...,...,...,...,...,...,...
413,3,1,NaN,0,0,8.0500,-1
414,1,-1,39.0,0,0,108.9000,0
415,3,1,38.5,0,0,7.2500,-1
416,3,1,NaN,0,0,8.0500,-1


FILLING NAN VALUES IN AGE COLUMN WITH CORRESSPONDING MEAN OF PASSENGER CLASS AND THE NAN VALUES IN EMBARKED AND FARE COLUMN WITH THEIR MEAN

In [16]:
training_data["Age"].fillna(training_data.groupby("Pclass")["Age"].transform("mean"),inplace=True)
training_data['Embarked'] = training_data['Embarked'].fillna(0)
testing_data["Age"].fillna(testing_data.groupby("Pclass")["Age"].transform("mean"),inplace=True)
testing_data["Fare"].fillna(testing_data["Fare"].mean(),inplace=True)

TRANSFORMING COLUMNS WITH BIGGER DATA VALUES INTO THE RANGE -3 TO 3

In [17]:
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
training_data[["Age","Fare"]] = sc.fit_transform(training_data[["Age","Fare"]])
testing_data[["Age","Fare"]] = sc.fit_transform(testing_data[["Age","Fare"]])

SPLITTING THE DATASET AND TRAINING THE LOGISTIC REGRESSION MODEL

In [18]:
Variables = training_data.drop("Survived",axis=1)
Output = training_data["Survived"]

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
X_train, X_val, y_train, y_val = train_test_split(Variables, Output, test_size=0.15, random_state=89)
lr.fit(X_train,y_train)
y_pred = lr.predict(X_val)

CHECKING THE FINAL ACCURACY

In [20]:
from sklearn.metrics import accuracy_score
accuracy_score(y_val,y_pred)

0.8432835820895522

CREATING FINAL SUBMISSION FILE

In [21]:
prediction=lr.predict(testing_data)

In [22]:
prediction

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,

In [23]:
Final_file = pd.DataFrame({
        "PassengerId": test_id,
        "Survived": prediction
    })

In [24]:
Final_file

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [25]:
Final_file.to_csv("Final_submission.csv",index=False)